# American Sign Language (ASL) Final Project W251 YOLOv5 Custom Dataset
---
 Adapted from Roboflow Collab located [here](https://colab.research.google.com/drive/1gDZ2xcTOgR39tGGs-EZ6i3RTs16wmzZQ)
 
 Used on a `g4dn.2xlarge` AWS EC2 Instance with Nvidia Deep Learning AMI installed and within a Nvidia docker container `nvcr.io/nvidia/pytorch:21.09-py3`
 
Command Line Scripts Executed:
 ```shell
 docker run --net=host --gpus=all --ipc=host -v ~/data:/data -p 8882:8888 -ti nvcr.io/nvidia/pytorch:21.09-py3 bash
 jupyter lab --no-browser --ip=0.0.0.0 --allow-root
 ```

#### 1. Download Yolov5 Repository & install any dependencies

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
#!git reset --hard 886f1c03d839575afecb059accf74296fad395b6 #we are using the latest repot version not this

Cloning into 'yolov5'...
remote: Enumerating objects: 11212, done.
remote: Total 11212 (delta 0), reused 0 (delta 0), pack-reused 11212
Receiving objects: 100% (11212/11212), 11.11 MiB | 43.77 MiB/s, done.
Resolving deltas: 100% (7759/7759), done.
/data/yolov5


In [1]:
# !cd /data/yolov5/ #assumes we are mounting to the data directory (need to change this if setup is different)
# !pip install -r requirements.txt  # install dependencies (ignore errors)
# !apt-get update # make sure system is updated and install python3-opencv
# !DEBIAN_FRONTEND=noninteractive apt-get install -y python3-opencv
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.10.0a0+3fd9dcf _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [2]:
#!pip install -q wandb
import wandb
# Logging Metrics using Weights and Biases
wandb.login()

wandb: Currently logged in as: ekhan (use `wandb login --relogin` to force relogin)


True

In [3]:
!wandb online

W&B online, running your script from this directory will now sync to the cloud.


#### 2. Make sure to download the dataset from Roboflow as YOLO v5 PyTorch

In [4]:
%cd /data/
!curl -L "https://app.roboflow.com/ds/n5fh1xx5Q8?key=g29NmS6OfD" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   888  100   888    0     0   2114      0 --:--:-- --:--:-- --:--:--  2114
100 35.7M  100 35.7M    0     0  45.9M      0 --:--:-- --:--:-- --:--:--  353M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/0-14_jpg.rf.f1d8473253d9e9f0162870bdde49fc75.jpg  
 extracting: test/images/0-17_jpg.rf.c6d591b9bb71adb4888f610670ee54e7.jpg  
 extracting: test/images/0-18_jpg.rf.fd6b1ce3afb1e946be06e15edc3aff34.jpg  
 extracting: test/images/0-34_jpg.rf.2ea18cc2be2893ca804e783329fd1959.jpg  
 extracting: test/images/0-37_jpg.rf.e3c633a82423b1c9b28016b37d7d7788.jpg  
 extracting: test/images/0-48_jpg.rf.eeee4f2efa5ea71b3b9f7d771bc6a8f0.jpg  
 extracting: test/images/0-50_jpg.rf.509

In [5]:
#!pip install roboflow
# !apt-get update
# !DEBIAN_FRONTEND=noninteractive apt-get install -y python3-opencv

#from roboflow import Roboflow
#rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

In [ ]:
#%mv valid yolov5

In [6]:
%cd /data/yolov5
%cat data.yaml

/data/yolov5
train: ../train/images
val: ../valid/images

nc: 35
names: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'four', 'hello', 'help', 'one', 'right', 'thanks', 'three', 'two', 'zero']

In [33]:
os.getcwd()

'/data/yolov5'

In [7]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [30]:
# cat the model used
# %cat /data/yolov5/models/yolov5s.yaml

In [8]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [9]:
%%writetemplate /data/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [11]:
import os
#import pandas as pd
# os.getcwd()
os.chdir('/data/yolov5/')

#### 3. Train Model on Custom Dataset

In [13]:
%%time
!python train.py --img 416 --batch 16 --epochs 300 --data data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

requirements: /data/yolov5/Pillow>=8.4.0 not found, check failed.
wandb: Currently logged in as: ekhan (use `wandb login --relogin` to force relogin)
train: weights=, cfg=./models/custom_yolov5s.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.1-21-ge6e36aa torch 1.10.0a0+3fd9dcf CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs

#### 5. Test model predictions

In [14]:
#os.chdir('/data/yolov5/')
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.1 --source ../test/images --project  yolov5

requirements: /data/yolov5/Pillow>=8.4.0 not found, check failed.
detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-21-ge6e36aa torch 1.10.0a0+3fd9dcf CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 232 layers, 7313943 parameters, 0 gradients, 17.0 GFLOPs
image 1/351 /data/test/images/0-14_jpg.rf.f1d8473253d9e9f0162870bdde49fc75.jpg: 416x416 1 K, Done. (0.010s)
image 2/351 /data/test/images/0-17_jpg.rf.c6d591b9bb71adb4888f610670ee54e7.jpg: 416x416 1 Q, Done. (0.010s)
image 3/351 /data/test/images/0-18_jpg.rf.fd6b1ce3afb1e946be06e15edc3aff34.jpg: 416x416 

In [75]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

#### 6. Export the best performing Pytorch model to a torchscript and tensorrt model

In [19]:
# export best model to tochscript and tensorrt
!python export.py --weights runs/train/yolov5s_results4/weights/best.pt --include torchscript engine --device 0

requirements: /data/yolov5/Pillow>=8.4.0 not found, check failed.
export: data=data/coco128.yaml, weights=['runs/train/yolov5s_results4/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=0, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'engine']
YOLOv5 🚀 v6.1-21-ge6e36aa torch 1.10.0a0+3fd9dcf CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 232 layers, 7338216 parameters, 0 gradients, 17.1 GFLOPs

PyTorch: starting from runs/train/yolov5s_results4/weights/best.pt with output shape (1, 25200, 40) (14.3 MB)

TorchScript: starting export with torch 1.10.0a0+3fd9dcf...
TorchScript: export success, saved as runs/train/yolov5s_results4/weights/best.torchscript (28.5 MB)

ONNX: starting export with onnx 1.8.204...
ONNX: export success, saved as runs/train/yolov5s

In [21]:
# rename .pt file to naming convention yolov5_best_300_epochs.pt 
#runs/train/yolov5s_results4/weights/best.pt
# !python export.py --weights runs/train/yolov5s_results4/weights/best.pt --include torchscript engine